# Example Model Explanations with Seldon

## Setup Seldon Core

Follow the instructions to [Setup Cluster](seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](seldon_core_setup.ipynb#Install-Seldon-Core).

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

## Running this notebook

For the [ImageNet Model](http://localhost:8888/notebooks/explainer_examples.ipynb#Imagenet-Model) you will need:

 - [alibi package](https://pypi.org/project/alibi/) (```pip install alibi```)
 
 This should install the required package dependencies, if not please also install:
 - [Pillow package](https://pypi.org/project/Pillow/) (```pip install Pillow```)
 - [matplotlib package](https://pypi.org/project/matplotlib/) (```pip install matplotlib```)
 - [tensorflow package](https://pypi.org/project/tensorflow/) (```pip install tensorflow```)

You will also need to start Jupyter with settings to allow for large payloads, for example:

```
jupyter notebook --NotebookApp.iopub_data_rate_limit=1000000000
```

## Income Prediction Model

In [ ]:
!pygmentize resources/income_explainer.yaml

In [ ]:
!kubectl apply -f resources/income_explainer.yaml

In [ ]:
!kubectl rollout status deploy/income-default-0

In [ ]:
!kubectl rollout status deploy/income-default-explainer

In [ ]:
from seldon_core.seldon_client import SeldonClient
import numpy as np
sc = SeldonClient(deployment_name="income",namespace="seldon")

In [ ]:
data = np.array([[39, 7, 1, 1, 1, 1, 4, 1, 2174, 0, 40, 9]])
r = sc.predict(gateway="ambassador",transport="rest",data=data)
print(r)

Use curl

In [ ]:
!curl -d '{"data": {"ndarray":[[39, 7, 1, 1, 1, 1, 4, 1, 2174, 0, 40, 9]]}}' \
   -X POST http://localhost:8003/seldon/seldon/income/api/v1.0/predictions \
   -H "Content-Type: application/json"

In [ ]:
data = np.array([[39, 7, 1, 1, 1, 1, 4, 1, 2174, 0, 40, 9]])
explanation = sc.explain(deployment_name="income",gateway="ambassador",transport="rest",data=data)
print(explanation)
assert(explanation.success==True)

Using curl

In [ ]:
!curl -d '{"data": {"ndarray":[[39, 7, 1, 1, 1, 1, 4, 1, 2174, 0, 40, 9]]}}' \
   -X POST http://localhost:8003/seldon/seldon/income-explainer/api/v1.0/explain \
   -H "Content-Type: application/json"

In [ ]:
!kubectl delete -f resources/income_explainer.yaml

## Movie Sentiment Model



In [ ]:
!pygmentize resources/moviesentiment_explainer.yaml

In [ ]:
!kubectl apply -f resources/moviesentiment_explainer.yaml

In [ ]:
!kubectl rollout status deploy/movie-default-0

In [ ]:
!kubectl rollout status deploy/movie-default-explainer

In [ ]:
from seldon_core.seldon_client import SeldonClient
import numpy as np
sc = SeldonClient(deployment_name="movie",namespace="seldon")

In [ ]:
data = np.array(['this film has great actors'])
r = sc.predict(gateway="ambassador",transport="rest",data=data,payload_type='ndarray')
print(r)
assert(r.success==True)

In [ ]:
data = np.array(['this film has great actors'])
explanation = sc.explain(deployment_name="movie",gateway="ambassador",transport="rest",data=data,payload_type='ndarray')
print(explanation)
assert(explanation.success==True)

In [ ]:
!kubectl delete -f resources/moviesentiment_explainer.yaml

## Imagenet Model



In [ ]:
!pygmentize resources/imagenet_explainer_grpc.yaml

In [ ]:
!kubectl apply -f resources/imagenet_explainer_grpc.yaml

In [ ]:
!kubectl rollout status deploy/image-default-0

In [ ]:
!kubectl rollout status deploy/image-default-explainer

In [ ]:
from PIL import Image
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.applications.inception_v3 import InceptionV3, decode_predictions
import alibi
from alibi.datasets import fetch_imagenet
import numpy as np

def get_image_data():
    data = []
    image_shape = (299, 299, 3)
    target_size = image_shape[:2]
    image = Image.open("cat-raw.jpg").convert('RGB')
    image = np.expand_dims(image.resize(target_size), axis=0)
    data.append(image)
    data = np.concatenate(data, axis=0)
    return data

data = get_image_data()

In [ ]:
from seldon_core.seldon_client import SeldonClient
import numpy as np
sc = SeldonClient(deployment_name="image",namespace="seldon",grpc_max_send_message_length= 27 * 1024 * 1024,grpc_max_receive_message_length= 27 * 1024 * 1024)

In [ ]:
import tensorflow as tf
data = get_image_data()
req = data[0:1]
r = sc.predict(gateway="ambassador",transport="grpc",data=req,payload_type='tftensor',client_return_type='proto')

preds = tf.make_ndarray(r.response.data.tftensor)

label = decode_predictions(preds, top=1)
plt.title(label[0])
plt.imshow(data[0])

In [ ]:
req = np.expand_dims(data[0], axis=0)
r = sc.explain(deployment_name="image",gateway="ambassador",transport="rest",data=req)
exp_arr = np.array(r.response['anchor'])

f, axarr = plt.subplots(1, 2)
axarr[0].imshow(data[0])
axarr[1].imshow(r.response['anchor'])
plt.show()


In [ ]:
!kubectl delete -f resources/imagenet_explainer_grpc.yaml